### PRE PROCESSING

In [3]:
import numpy as np 
import pandas as pd 
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
store_df=pd.read_csv("store.csv")
print(store_df.shape)
store_df.head()

(1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
training=pd.read_csv("train.csv")
print(training.shape)
training.head()
training.describe()

(1017209, 9)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


In [6]:
# training['Sales_per_customer']=training['Sales']/training['Customers']
# training['Sales_per_customer'].describe() 

In [7]:
dates_data=pd.read_csv("test.csv")
print(dates_data.shape)
dates_data.head()
# date column month and year
training['Year']  = training['Date'].apply(lambda x: int(str(x)[:4]))
training['Month'] = training['Date'].apply(lambda x: int(str(x)[5:7]))

dates_data['Year']  = dates_data['Date'].apply(lambda x: int(str(x)[:4]))
dates_data['Month'] = dates_data['Date'].apply(lambda x: int(str(x)[5:7]))

# StateHoliday has values 0 & "0", So, we need to merge values with 0 to "0"
training["StateHoliday"]= training["StateHoliday"].replace(0, "0")


(41088, 8)


In [8]:
# Merging train and store_df
training_store = pd.merge(right=store_df,left=training,how='outer',on='Store')
print(training_store.shape)
training_store.head()
training_store.info()

(1017209, 20)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 20 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   Year                       1017209 non-null  int64  
 10  Month                      1017209 non-null  int64  
 11  StoreType                  1017209 non-null  object 
 12  Assortment                 1017209 non-null  object 
 13

In [9]:
training_store.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
Year                              0
Month                             0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### Treating missing and null value


In [10]:
# treating missing values with median of their respective columns and dropping PromoInterval as it is not that much related to the dependent variable 

median_sales = training_store['Sales'].astype('float').median(axis=0)
training_store['Sales'].replace(np.nan,median_sales,inplace=True)

median_customers = training_store['Customers'].astype('float').median(axis=0)
training_store['Customers'].replace(np.nan,median_customers,inplace=True)


median_comp_dist = training_store['CompetitionDistance'].astype('float').median(axis=0)
training_store['CompetitionDistance'].replace(np.nan,math.floor(median_comp_dist),inplace=True)

median_comp_month = training_store['CompetitionOpenSinceMonth'].astype('float').median(axis=0)
training_store['CompetitionOpenSinceMonth'].replace(np.nan,math.floor(median_comp_month),inplace=True)

median_comp_year = training_store['CompetitionOpenSinceYear'].astype('float').median(axis=0)
training_store['CompetitionOpenSinceYear'].replace(np.nan,math.floor(median_comp_year),inplace=True)

median_promo2_week = training_store['Promo2SinceWeek'].astype('float').median(axis=0)
training_store['Promo2SinceWeek'].replace(np.nan,math.floor(median_promo2_week),inplace=True)

median_promo2_year = training_store['Promo2SinceYear'].astype('float').median(axis=0)
training_store['Promo2SinceYear'].replace(np.nan,math.floor(median_promo2_year),inplace=True)

training_store.drop(['PromoInterval'],axis=1,inplace=True)

In [11]:
training_store.isnull().sum() 

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
Year                         0
Month                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
dtype: int64

### test

In [12]:
dates_data.head(4)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month
0,1,1,4,2015-09-17,1.0,1,0,0,2015,9
1,2,3,4,2015-09-17,1.0,1,0,0,2015,9
2,3,7,4,2015-09-17,1.0,1,0,0,2015,9
3,4,8,4,2015-09-17,1.0,1,0,0,2015,9


In [13]:
training_store.head(6)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,c,a,1270.0,9.0,2008.0,0,22.0,2012.0
1,1,4,2015-07-30,5020,546,1,1,0,1,2015,7,c,a,1270.0,9.0,2008.0,0,22.0,2012.0
2,1,3,2015-07-29,4782,523,1,1,0,1,2015,7,c,a,1270.0,9.0,2008.0,0,22.0,2012.0
3,1,2,2015-07-28,5011,560,1,1,0,1,2015,7,c,a,1270.0,9.0,2008.0,0,22.0,2012.0
4,1,1,2015-07-27,6102,612,1,1,0,1,2015,7,c,a,1270.0,9.0,2008.0,0,22.0,2012.0
5,1,7,2015-07-26,0,0,0,0,0,0,2015,7,c,a,1270.0,9.0,2008.0,0,22.0,2012.0


In [24]:
cleanup = {"StoreType": {"a":0, "b":1, "c":2,"d":3}}
training_store.replace(cleanup, inplace=True)
training_store.head()
# training_store["StoreType"]
training_store["StoreType"].unique()
training_store["StateHoliday"].unique()

array(['0', 'a', 'b', 'c'], dtype=object)

In [19]:
training_store["Assortment"].unique()

array(['a', 'c', 'b'], dtype=object)

In [68]:
# Create dummy varibales for DayOfWeek
training_dummies  = pd.get_dummies(training['DayOfWeek'], prefix='Day')
# training_dummies.drop(['Day_7'], axis=1, inplace=True)

testing_dummies = pd.get_dummies(dates_data['DayOfWeek'],prefix='Day')
# testing_dummies.drop(['Day_7'], axis=1, inplace=True)

training = training.join(training_dummies)
dates_data = dates_data.join(testing_dummies)

In [69]:
# 
dates_data[dates_data.isnull().any(axis=1)]
dates_data["Open"].fillna(1, inplace = True) 
dates_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41088 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
 8   Year           41088 non-null  int64  
 9   Month          41088 non-null  int64  
 10  Day_1          41088 non-null  uint8  
 11  Day_2          41088 non-null  uint8  
 12  Day_3          41088 non-null  uint8  
 13  Day_4          41088 non-null  uint8  
 14  Day_5          41088 non-null  uint8  
 15  Day_6          41088 non-null  uint8  
 16  Day_7          41088 non-null  uint8  
dtypes: float64(1), int64(7), object(2), uint8(7)
memor

In [44]:
# remove all rows(store,date) that were closed
training= training[training["Open"] != 0]

In [72]:
# putting id=0 if store is closed
store_close_id = dates_data["Id"][dates_data["Open"] == 0].values
print(store_close_id.shape)
store_close_id

(5984,)


array([  544,   677,   841, ..., 40230, 40231, 40232])

In [75]:
# remove all rows(store,date) that were closed
dates_data= dates_data[dates_data["Open"] != 0]

In [76]:
dates_data = dates_data.reset_index()

In [77]:
dates_data.drop(["index"],axis =1, inplace= True)

In [80]:
training.drop(["Date"],axis = 1, inplace=True)
training.head()

,Store,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7
0,1,5263,555,1,1,0,1,2015,7,0,0,0,0,1,0,0
1,2,6064,625,1,1,0,1,2015,7,0,0,0,0,1,0,0
2,3,8314,821,1,1,0,1,2015,7,0,0,0,0,1,0,0
3,4,13995,1498,1,1,0,1,2015,7,0,0,0,0,1,0,0
4,5,4822,559,1,1,0,1,2015,7,0,0,0,0,1,0,0


In [79]:
dates_data.head()

,Id,Store,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7
0,1,1,2015-09-17,1.0,1,0,0,2015,9,0,0,0,1,0,0,0
1,2,3,2015-09-17,1.0,1,0,0,2015,9,0,0,0,1,0,0,0
2,3,7,2015-09-17,1.0,1,0,0,2015,9,0,0,0,1,0,0,0
3,4,8,2015-09-17,1.0,1,0,0,2015,9,0,0,0,1,0,0,0
4,5,9,2015-09-17,1.0,1,0,0,2015,9,0,0,0,1,0,0,0
